# Training of the model step-by-step

## Import packages

In [ ]:
import os
import argparse
import numpy as np
import joblib
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import h5py as h5
from sklearn.preprocessing import MinMaxScaler
from gwpy.timeseries import TimeSeries

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, TimeDistributed, RepeatVector, Conv1D,\
                                    MaxPooling1D, UpSampling1D, Flatten, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow.keras.regularizers as regularizers

sns.set(color_codes=True)

## Define the models

In [ ]:
def autoencoder_LSTM(X):
    inputs = Input(shape=(X.shape[1], X.shape[2]))
    L1 = LSTM(32, activation='relu', return_sequences=True, 
              kernel_regularizer=regularizers.l2(0.00))(inputs)
    L2 = LSTM(8, activation='relu', return_sequences=False)(L1)
    L3 = RepeatVector(X.shape[1])(L2)
    L4 = LSTM(8, activation='relu', return_sequences=True)(L3)
    L5 = LSTM(32, activation='relu', return_sequences=True)(L4)
    output = TimeDistributed(Dense(X.shape[2]))(L5)    
    model = Model(inputs=inputs, outputs=output)
    return model

def autoencoder_GRU(X):
    inputs = Input(shape=(X.shape[1], X.shape[2]))
    L1 = GRU(32, activation='relu', return_sequences=True, 
              kernel_regularizer=regularizers.l2(0.00))(inputs)
    L2 = GRU(8, activation='relu', return_sequences=False)(L1)
    L3 = RepeatVector(X.shape[1])(L2)
    L4 = GRU(8, activation='relu', return_sequences=True)(L3)
    L5 = GRU(32, activation='relu', return_sequences=True)(L4)
    output = TimeDistributed(Dense(X.shape[2]))(L5)    
    model = Model(inputs=inputs, outputs=output)
    return model

def autoencoder_Conv(X):
    inputs = Input(shape=(X.shape[1], X.shape[2]))
    L1 = Conv1D(16, 3, activation="relu", padding="same")(inputs) # 10 dims
    #x = BatchNormalization()(x)
    L2 = MaxPooling1D(4, padding="same")(L1) # 5 dims
    L3 = Conv1D(10, 3, activation="relu", padding="same")(L2) # 5 dims
    #x = BatchNormalization()(x)
    encoded = MaxPooling1D(4, padding="same")(L3) # 3 dims
    # 3 dimensions in the encoded layer
    L4 = Conv1D(10, 3, activation="relu", padding="same")(encoded) # 3 dims
    #x = BatchNormalization()(x)
    L5 = UpSampling1D(4)(L4) # 6 dims
    L6 = Conv1D(16, 2, activation='relu')(L5) # 5 dims
    #x = BatchNormalization()(x)
    L7 = UpSampling1D(4)(L6) # 10 dims
    output = Conv1D(1, 3, activation='sigmoid', padding='same')(L7) # 10 dims
    model = Model(inputs=inputs, outputs = output)
    return model

def autoencoder_Conv2(X):
    inputs = Input(shape=(X.shape[1], X.shape[2]))
    L1 = Conv1D(16, 4, activation="relu", dilation_rate=1, padding="same")(inputs)
    L2 = MaxPooling1D(2)(L1)
    L3 = Conv1D(32, 4, activation="relu", dilation_rate=2, padding="same")(L2)
    L4 = MaxPooling1D(2)(L3) 
    L5 = Conv1D(64, 4, activation="relu", dilation_rate=2, padding="same")(L4)
    L6 = MaxPooling1D(4)(L5)
    L7 = Conv1D(128, 8, activation="relu", dilation_rate=2, padding="same")(L6)
    encoded = MaxPooling1D(4)(L7)
    L7 = Conv1D(128, 8, activation="relu", dilation_rate=2, padding="same")(encoded)
    L8 = UpSampling1D(4)(L7)
    L9 = Conv1D(64, 4, activation="relu", dilation_rate=2, padding="same")(L8)
    L10 = UpSampling1D(4)(L9)
    L11 = Conv1D(32, 4, activation="relu", dilation_rate=2, padding="same")(L10)
    L12 = UpSampling1D(4)(L11)
    L13 = Conv1D(16, 3, activation="relu", dilation_rate=1, padding="same")(L12)
    L14 = UpSampling1D(2)(L13)
    output = Conv1D(1, 4, activation="relu", dilation_rate=1, padding="same")(L12)
    model = Model(inputs=inputs, outputs = output)
    return model

def autoencoder_ConvDNN(X):
    inputs = Input(shape=(X.shape[1], X.shape[2]))
    L1 = Conv1D(16, 3, activation="relu", padding="same")(inputs) # 10 dims
    #x = BatchNormalization()(x)
    L2 = MaxPooling1D(4, padding="same")(L1) # 5 dims
    L3 = Conv1D(10, 3, activation="relu", padding="same")(L2) # 5 dims
    #x = BatchNormalization()(x)
    encoded = MaxPooling1D(4, padding="same")(L3) # 3 dims
    x = Flatten()(encoded)
    x = Dense(30, activation='relu')(x)
    x = Dense(10, activation='relu')(x)
    x = Dense(30, activation='relu')(x)
    x = Dense(70, activation='relu')(x)
    x = Reshape((7, 10))(x)
    # 3 dimensions in the encoded layer
    L4 = Conv1D(10, 3, activation="relu", padding="same")(x) # 3 dims
    #x = BatchNormalization()(x)
    L5 = UpSampling1D(4)(L4) # 6 dims
    L6 = Conv1D(16, 2, activation='relu')(L5) # 5 dims
    #x = BatchNormalization()(x)
    L7 = UpSampling1D(4)(L6) # 10 dims
    output = Conv1D(1, 3, activation='sigmoid', padding='same')(L7) # 10 dims
    model = Model(inputs=inputs, outputs = output)
    return model

def autoencoder_ConvLSTM(X):
    inputs = Input(shape=(X.shape[1], X.shape[2]))
    L1 = Conv1D(16, 3, activation="relu", padding="same")(inputs) # 10 dims
    #x = BatchNormalization()(x)
    L2 = MaxPooling1D(4, padding="same")(L1) # 5 dims
    L3 = Conv1D(10, 3, activation="relu", padding="same")(L2) # 5 dims
    #x = BatchNormalization()(x)
    encoded = MaxPooling1D(4, padding="same")(L3) # 3 dims
    x = Reshape((70, 1))(encoded)
    
    x = LSTM(32, activation='relu', return_sequences=False, 
              kernel_regularizer=regularizers.l2(0.00))(x)
    x = RepeatVector(70)(x)
    x = LSTM(32, activation='relu', return_sequences=True)(x)
    out = TimeDistributed(Dense(1))(x)  
    
    x = Reshape((7, 10))(out)
    # 3 dimensions in the encoded layer
    L4 = Conv1D(10, 3, activation="relu", padding="same")(x) # 3 dims
    #x = BatchNormalization()(x)
    L5 = UpSampling1D(4)(L4) # 6 dims
    L6 = Conv1D(32, 2, activation='relu')(L5) # 5 dims
    #x = BatchNormalization()(x)
    L7 = UpSampling1D(4)(L6) # 10 dims
    output = Conv1D(1, 3, activation='sigmoid', padding='same')(L7) # 10 dims
    model = Model(inputs=inputs, outputs = output)
    return model

def autoencoder_DNN(X):
    inputs = Input(shape=(X.shape[1], X.shape[2]))
    x = Flatten()(inputs)
    x = Dense(int(X.shape[1]/2), activation='relu')(x)
    x = Dense(int(X.shape[1]/10), activation='relu')(x)
    x = Dense(int(X.shape[1]/2), activation='relu')(x)
    x = Dense(X.shape[1], activation='relu')(x)
    output = Reshape((X.shape[1], 1))(x)
    model = Model(inputs=inputs, outputs=output)
    return model

## Define some I/O variables

1. Select one of the above defined models (autoencoder_LSTM, autoencoder_GRU, autoencoder_Conv, autoencoder_Conv2, autoencoder_ConvDNN, autoencoder_ConvLSTM, autoencoder_DNN).
2. Select the LIGO detector, the sampling frequency of the detector, whether to filter the data and the # of timesteps to pass to the model.


In [ ]:
%%capture

model_chosen = 'autoencoder_LSTM'   #change this
outdir = './output/' + model_chosen
detector = 'L1'                     # LIGO detector. L1 or H1
freq = '2'                          # Sampling frequency of the detector in KHz. 2 or 4
filtered = '1'                      # Apply bandpass and whitening filters. 1/0 
timesteps = 100                     # Number of timesteps passed to model

# Create output directory
os.system(f'mkdir {outdir}')

# Define frequency in Hz instead of KHz
if int(freq) == 2:
    freq = 2048
elif int(freq) == 4:
    freq = 4096
else:
    print(f'Given frequency {freq}kHz is not supported. Correct values are 2 or 4kHz.')

## Load and explore the dataset

In [ ]:
# print(os.getcwd())

load = h5.File('./../dataset/dataset1.hdf', 'r')   # Change this to dataset location!
print('File keys:\n')
print([key for key in load.keys()])

group = load['noise_samples']
print('\n'+'Noise_samples keys:\n')
print([key for key in group.keys()])

dataset = group[f'{str(detector).lower()}_strain'][0:6000]  # I use only one half of the dataset
print('\n'+'L1 strain shape:\n')
print(dataset.shape)

### Strain plot example

In [ ]:
y = group['l1_strain'][0]
print(y.shape)
plt.plot(y[0:200]);

## Pre-processing

### Apply whitening and bandpass

With LIGO simulated data, the sample isn't pre-filtered so need to filter.

(**???** The gwpy package generates filtered data.
In particular it applies a whitening procedure and a bandpass, **but** it uses different lower and upper bounds (20 Hz & 2048 Hz - see the file `waveform_params.ini`)).

Anyway, here I apply the filtering function.

In [ ]:
def filters(array, sample_frequency):
    """ Apply preprocessing such as whitening and bandpass """
    strain = TimeSeries(array, sample_rate=int(sample_frequency))   # Create an instance of the class TimeSeries
    white_data = strain.whiten(fftlength=4, fduration=4)            # Whiten the TimeSeries. fftlength = fft integration length in s. fduration = duration in s of the filter
    bp_data = white_data.bandpass(50, 250)                          # Band-pass filter
    return bp_data.value                                            # .value is the numerical value of the instance

if bool(int(filtered)):
    print('Filtering data with whitening and bandpass')
    print(f'Sample Frequency: {freq} Hz')
    x = [filters(sample, freq) for sample in dataset]
    print('Filtering completed')

### Normalize the data

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(x)
scaler_filename = f"{outdir}/scaler_data_{detector}"
joblib.dump(scaler, scaler_filename)                      # Joblib.dump persists scaler into the file scaler_filename

 ### Data augmentation
 
This passage extends the dataset and is needed if there's not enough data.
Here I skip it, because of hardware constraint.

In [ ]:
def augmentation(X_train, timesteps):
    x = []
    for sample in X_train:
        if sample.shape[0] % timesteps != 0:
            corrected_sample = sample[:-1 * int(sample.shape[0] % timesteps)]    # Cut off the tail
        sliding_sample = np.array([corrected_sample[i:i + timesteps][:] for i in [int(timesteps / 2) * n for n in range(int(len(corrected_sample) / (timesteps / 2)) - 1)]]) # window (width=timesteps) sliding on the cut sample
        x.append(sliding_sample)
    return np.array(x)

# X_train = augmentation(X_train, timesteps)  

### Trim and reshape

Trim the dataset to be batch-friendly and reshape it into timestep format.

In [ ]:
#X_train[0].shape[0]
print(f'Current dataset shape: {X_train.shape}')

In [ ]:
x = []
for event in range(len(X_train)):
    if X_train[event].shape[0] % timesteps != 0:
        x.append(X_train[event][:-1 * int(X_train[event].shape[0] % timesteps)])
X_train = np.array(x)

In [ ]:
print(f'Dataset shape after trimming: {X_train.shape}')

Reshape inputs for LSTM [samples, timesteps, features]

In [1]:
X_train = X_train.reshape(-1, timesteps, 1)   # Reshape the array w/o changing its data. The value for -1 is inferred form the other dimensions
print("Training data shape:", X_train.shape)

NameError: name 'X_train' is not defined

## Define and train the model

In [ ]:
model = autoencoder_LSTM(X_train)
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
nb_epochs = 10    # This was 300 in the original code!
batch_size = 1024
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint(f'{outdir}/best_model.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, X_train, epochs=nb_epochs, batch_size=batch_size,
                        validation_split=0.2, callbacks=[early_stop, mcp_save]).history
model.save(f'{outdir}/last_model.hdf5')

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6), dpi=80)
ax.plot(history['loss'], 'b', label='Train', linewidth=2)
ax.plot(history['val_loss'], 'r', label='Validation', linewidth=2)
ax.set_title('Model loss', fontsize=16)
ax.set_ylabel('Loss (mse)')
ax.set_xlabel('Epoch')
ax.legend(loc='upper right')
plt.show()
plt.savefig(f'{outdir}/loss.jpg')